The ultimate objective of RQ2 is whether Arachne can generate patches that fix the "given" misbehaviour of a DNN. In the traditional APR with generate-and-validate (G&V) approach, we don't consider whether the current fixes can fix the unseen program failure that may take place in future, in general. Similarly, here, we want to investigate the pure capability of Arachne to adjust model not to have the given misbehaviour. However, since the inherent nature of DNNs and that of general software programs are different, we also look into the "generality" of the patches, which takes place in RQ3 (not in RQ2). 

Question to answer here: can arachne generate patches directly applied to the DNN's neural weights for the observed misclassification? In other words, here, we investigate the pure capability of Arahcne adjusting the trained model not to have certain misbehaviour. 
(RQ3 will then examine whether Arachne can generat patches using unseen inputs and whether these synethesised patches can be generalisable to inputs unseen during the training and also during the patch generation? In other words, is Arachne generasible to completely unseen inputs?  


### Overall Repair Rate: a repair rate over the entire faulty inputs
### Target Repair Rate: a repair rate over the sampled 10%


In [1]:
import os, sys
sys.path.insert(0, "../")
import utils.data_util as data_util

In [3]:
import json
import numpy as np
import subprocess
import glob
import pandas as pd
from tqdm import tqdm

In [8]:
# change to "results" to "/volume1/coinse/Arachne/results/rev"
test_resultdir = "/Users/jeongju.sohn/workdir/arachne/arachne/results/rq2/on_test"

# train_resultdir -> contains the results when we run the experiments with the past setting 
#(i.e., repair using the training ddata)
train_resultdir = "/Users/jeongju.sohn/workdir/arachne/arachne/results/rq2/on_train"
indexdir = "/Users/jeongju.sohn/workdir/arachne/arachne/indices"

In [9]:
def get_init_pred_file(which_data, on_test = True):
    """
    """
    if which_data == "fashion_mnist":
        return os.path.join(indexdir, 
                            "fm/test/fashion_mnist.init_pred.indices.csv" 
                            if on_test else "fm/fashion_mnist.init_pred.indices.csv")
    elif which_data == 'cifar10':
        return os.path.join(indexdir, 
                            "cm/test/cifar10.init_pred.indices.csv" 
                            if on_test else "cm/cifar10.init_pred.indices.csv" )
    elif which_data == 'GTSRB':
        return os.path.join(indexdir, 
                            "GTSRB/rq1/wo_hist/test/GTSRB.init_pred.indices.csv" 
                            if on_test else "GTSRB/rq1/wo_hist/GTSRB.init_pred.indices.csv")
    else:# imdb
        print ("Wrong data: {}".format(which_data))

In [10]:
def get_predictions(resultdir, which_data, loc, loc_abbr, seeds):
    """
    """
    assert loc in ['localiser', 'gradient_loss', 'random'], loc

    if which_data == 'fashion_mnist':
        datadir = os.path.join(resultdir, "fm/{}/pred".format(loc))     
    elif which_data == 'cifar10':
        datadir = os.path.join(resultdir, "cm/{}/pred".format(loc))
    else: #if which_data == 'GTSRB':
        datadir = os.path.join(resultdir, "gtsrb/{}/pred".format(loc)) 
        
    predictions = {}
    for seed in seeds:
        #if not on_eval:
        fpath = os.path.join(datadir, "pred.{}.{}.train.pkl".format(loc_abbr, seed))
        #else:
        #    assert 'train' in resultdir, resultdir
        #    fpath = os.path.join(datadir, "pred.{}.{}.eval.pkl".format(loc_abbr, seed)) 
        predictions[seed] = pd.read_pickle(fpath)        
    
    return predictions

In [28]:
def compute_br_rr(result_df, seed, index_file):
    """
    broken: initally correct -> incorrect
    repair: initially incorrect -> correct
    """
    # get the indices to the initally targeted incorrect input
    indices_to_targeted = data_util.get_misclf_for_rq2(index_file, percent = 0.1, seed = seed)
    
    num_data = len(result_df)
    init_incorrect_df = result_df.loc[result_df.init_flag == False]
    init_correct_df = result_df.loc[result_df.init_flag == True]
    init_num_incorrect = len(init_incorrect_df)
    init_num_correct = len(init_correct_df)
    
    patched_df = init_incorrect_df.loc[init_incorrect_df.true == init_incorrect_df.new_pred]
    broken_df = init_correct_df.loc[init_correct_df.true != init_correct_df.new_pred]
    
    # repair rate -> overall needed b/c we sample 10\% of the entire misclassified inputs
    overall_rr = len(patched_df)/init_num_incorrect
    
    ## only for the targeted
    target_patched_df = patched_df.loc[patched_df.index.isin(indices_to_targeted)]
    target_rr = len(target_patched_df)/len(indices_to_targeted)
    
    # broken rate
    br = len(broken_df)/init_num_correct
    
    # additional
    init_acc = np.sum(result_df.true == result_df.pred)/len(result_df)
    aft_acc = np.sum(result_df.true == result_df.new_pred)/len(result_df)
    acc_r = aft_acc/init_acc
    
    return {'overall_rr':overall_rr, 'target_rr':target_rr, 'br':br, 'acc_r':acc_r}

In [29]:
def main_compute_br_rr(resultdir, which_data, loc, seeds = np.arange(30)):
    """
    """
    if loc in 'localiser':
        loc_abbr = 'loc'
    elif loc in 'gradient_loss':
        loc_abbr = 'gl'
    else:
        loc_abbr = 'rd'

    predictions_df = get_predictions(resultdir, which_data, loc, loc_abbr, seeds)
    index_file = get_init_pred_file(which_data, on_test = 'test' in resultdir)
    
    rates_of_all = {}
    for seed in tqdm(seeds):
        rates = compute_br_rr(predictions_df[seed], seed, index_file)
        rates_of_all[seed] = rates
    
    return pd.DataFrame(rates_of_all).T

In [30]:
def analy_br_rr(overall_rates_df, decimals = 3):
    """
    compute the average br and rr
    """
    avgs = overall_rates_df.mean().to_dict()
        
    print ("Overall RR:{}, Target RR:{}, Broken Rate: {}, Acc Rate: {}".format(
        np.round(avgs['overall_rr'], decimals = decimals),
        np.round(avgs['target_rr'], decimals = decimals),
        np.round(avgs['br'], decimals = decimals), 
        np.round(avgs['acc_r'], decimals = decimals)))
    
    #print (np.round(avgs['overall_rr'], decimals = decimals), 
    #       np.round(avgs['target_rr'], decimals = decimals),
    #       np.round(avgs['br'], decimals = decimals))
    return avgs

# Evaluation

In [4]:
seeds = np.arange(30)

## Fashion MNIST

In [54]:
tst_fm_rates_of_loc = main_compute_br_rr(test_resultdir, 'fashion_mnist', 'localiser', seeds = seeds)
tst_fm_rates_of_gl = main_compute_br_rr(test_resultdir, 'fashion_mnist', 'gradient_loss', seeds = seeds)
tst_fm_rates_of_rd = main_compute_br_rr(test_resultdir, 'fashion_mnist', 'random', seeds = seeds)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 112.65it/s]


In [68]:
train_fm_rates_of_loc = main_compute_br_rr(train_resultdir, 'fashion_mnist', 'localiser', seeds = seeds)
train_fm_rates_of_gl = main_compute_br_rr(train_resultdir, 'fashion_mnist', 'gradient_loss', seeds = seeds)
train_fm_rates_of_rd = main_compute_br_rr(train_resultdir, 'fashion_mnist', 'random', seeds = seeds)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 51.37it/s]


In [55]:
print ("Localiser")
_ = analy_br_rr(tst_fm_rates_of_loc, decimals = 4)
print ("\nGradient loss")
_ = analy_br_rr(tst_fm_rates_of_gl, decimals = 4)
print ("\nRandom")
_ = analy_br_rr(tst_fm_rates_of_rd, decimals = 4)

Localiser
Overall RR:0.0423, Target RR:0.0866, Broken Rate: 0.007, Acc Rate: 0.9981

Gradient loss
Overall RR:0.0473, Target RR:0.095, Broken Rate: 0.0079, Acc Rate: 0.9978

Random
Overall RR:0.0206, Target RR:0.0542, Broken Rate: 0.0032, Acc Rate: 0.9993


In [69]:
print ("Localiser")
_ = analy_br_rr(train_fm_rates_of_loc, decimals = 4)
print ("\nGradient loss")
_ = analy_br_rr(train_fm_rates_of_gl, decimals = 4)
print ("\nRandom")
_ = analy_br_rr(train_fm_rates_of_rd, decimals = 4)

Localiser
Overall RR:0.1126, Target RR:0.135, Broken Rate: 0.0068, Acc Rate: 0.9983

Gradient loss
Overall RR:0.131, Target RR:0.1503, Broken Rate: 0.0103, Acc Rate: 0.9956

Random
Overall RR:0.0311, Target RR:0.0552, Broken Rate: 0.0012, Acc Rate: 1.0002


## CIFAR-10

In [56]:
tst_c10_rates_of_loc = main_compute_br_rr(test_resultdir, 'cifar10', 'localiser', seeds = np.arange(30))
tst_c10_rates_of_gl = main_compute_br_rr(test_resultdir, 'cifar10', 'gradient_loss', seeds = np.arange(30))
tst_c10_rates_of_rd = main_compute_br_rr(test_resultdir, 'cifar10', 'random', seeds = np.arange(30))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 117.15it/s]


In [66]:
train_c10_rates_of_loc = main_compute_br_rr(train_resultdir, 'cifar10', 'localiser', seeds = np.arange(30))
train_c10_rates_of_gl = main_compute_br_rr(train_resultdir, 'cifar10', 'gradient_loss', seeds = np.arange(30))
train_c10_rates_of_rd = main_compute_br_rr(train_resultdir, 'cifar10', 'random', seeds = np.arange(30))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 53.56it/s]


In [57]:
print ("Localiser")
_ = analy_br_rr(tst_c10_rates_of_loc, decimals = 4)
print ("\nGradient loss")
_ = analy_br_rr(tst_c10_rates_of_gl, decimals = 4)
print ("\nRandom")
_ = analy_br_rr(tst_c10_rates_of_rd, decimals = 4)

Localiser
Overall RR:0.0742, Target RR:0.1083, Broken Rate: 0.0305, Acc Rate: 0.998

Gradient loss
Overall RR:0.1169, Target RR:0.1357, Broken Rate: 0.0593, Acc Rate: 0.9857

Random
Overall RR:0.0066, Target RR:0.0174, Broken Rate: 0.0013, Acc Rate: 1.0013


In [67]:
print ("Localiser")
_ = analy_br_rr(train_c10_rates_of_loc, decimals = 4)
print ("\nGradient loss")
_ = analy_br_rr(train_c10_rates_of_gl, decimals = 4)
print ("\nRandom")
_ = analy_br_rr(train_c10_rates_of_rd, decimals = 4)

Localiser
Overall RR:0.1278, Target RR:0.1463, Broken Rate: 0.0104, Acc Rate: 1.0028

Gradient loss
Overall RR:0.1597, Target RR:0.1673, Broken Rate: 0.0251, Acc Rate: 0.9914

Random
Overall RR:0.0148, Target RR:0.0289, Broken Rate: 0.0008, Acc Rate: 1.0007


## GTSRB

In [63]:
tst_gtsrb_rates_of_loc = main_compute_br_rr(test_resultdir, 'GTSRB', 'localiser', seeds = np.arange(30))
tst_gtsrb_rates_of_gl = main_compute_br_rr(test_resultdir, 'GTSRB', 'gradient_loss', seeds = np.arange(30))
tst_gtsrb_rates_of_rd = main_compute_br_rr(test_resultdir, 'GTSRB', 'random', seeds = np.arange(30))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 112.43it/s]


In [64]:
train_gtsrb_rates_of_loc = main_compute_br_rr(train_resultdir, 'GTSRB', 'localiser', seeds = np.arange(30))
train_gtsrb_rates_of_gl = main_compute_br_rr(train_resultdir, 'GTSRB', 'gradient_loss', seeds = np.arange(30))
train_gtsrb_rates_of_rd = main_compute_br_rr(train_resultdir, 'GTSRB', 'random', seeds = np.arange(30))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 50.75it/s]


In [61]:
print ("Localiser")
_ = analy_br_rr(tst_gtsrb_rates_of_loc, decimals = 4)
print ("\nGradient loss")
_ = analy_br_rr(tst_gtsrb_rates_of_gl, decimals = 4)
print ("\nRandom")
_ = analy_br_rr(tst_gtsrb_rates_of_rd, decimals = 4)

Localiser
Overall RR:0.0692, Target RR:0.1304, Broken Rate: 0.007, Acc Rate: 1.0007

Gradient loss
Overall RR:0.0683, Target RR:0.1067, Broken Rate: 0.0082, Acc Rate: 0.9993

Random
Overall RR:0.0039, Target RR:0.0093, Broken Rate: 0.0, Acc Rate: 1.0004


In [65]:
print ("Localiser")
_ = analy_br_rr(train_gtsrb_rates_of_loc, decimals = 4)
print ("\nGradient loss")
_ = analy_br_rr(train_gtsrb_rates_of_gl, decimals = 4)
print ("\nRandom")
_ = analy_br_rr(train_gtsrb_rates_of_rd, decimals = 4)

Localiser
Overall RR:0.75, Target RR:0.7, Broken Rate: 0.0, Acc Rate: 1.0001

Gradient loss
Overall RR:0.7083, Target RR:0.6667, Broken Rate: 0.0001, Acc Rate: 1.0

Random
Overall RR:0.0083, Target RR:0.0, Broken Rate: 0.0, Acc Rate: 1.0


Overall, I think these results can be used to suppor the claim that Arachne can localise and fix, 
then, in RQ3, emphasise that the main target or the main goal of Arachne is to generate hot-fixes of a specific type ..
+ think taking only 10% may affect here...
+ did I take the same number of pos? (anw:  for loc)